<!-- Badges: -->
[![Donate](https://img.shields.io/badge/Donate-PayPal-green.svg?logo=paypal&style=flat-square)](https://www.paypal.me/CamponogaraViera/100)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/simulation.ipynb)
[![License](https://img.shields.io/github/license/QuCAI-Lab/ibm2021-open-science-prize.svg?logo=CreativeCommons&style=flat-square)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/LICENSE.md)
[![Contributions](https://img.shields.io/badge/contributions-welcome-orange?style=flat-square)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/pulls)
[![Maintenance](https://img.shields.io/badge/Maintained%3F-yes-green.svg)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/graphs/commit-activity)
[![Release](https://img.shields.io/github/release/QuCAI-Lab/ibm2021-open-science-prize.svg)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/releases)

<div align="center">
  <h1> <a href="https://research.ibm.com/blog/quantum-open-science-prize">2021 IBM Open Science Prize</a>  </h1>
  <h1><b> Supplementary Material </b></h1>
  <h3><b> Simulating the XXX Heisenberg Model Hamiltonian for a System of Three Interacting Spin-1/2 Particles on IBM Quantum’s 7-qubit Jakarta Processor </b></h3>
</div>
<br> 

<center> <b>Author: ¹Lucas Camponogara Viera</b></center>
<center>
<b><a target="_blank" href="https://en.ntnu.edu.tw/">¹National Taiwan Normal University - NTNU, Taipei, Taiwan</a></b>.
</center>

<br>

[comment]: <> ()
<table class="tfo-notebook-buttons" align="head">
  <td>
    <a target="_blank" href="https://github.com/QuCAI-Lab/ibm2021-open-science-prize"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
</table>

<center><a href="https://github.com/QuCAI-Lab/ibm2021-open-science-prize"><b>View on GitHub</b></a></center>

    


In [1]:
%autosave 30

Autosaving every 30 seconds


---
In this Jupyter notebook, the reader will find some of the equations of the quantum simulation presented in the `simulation.ipynb` notebook file. The mathematical framework employed in the remainder of this Jupyter notebook is the contemporary matrix mechanics formalism of quantum mechanics initially developed by Werner Heisenberg, Max Born, and Pascual Jordan in a sequel of articles between 1925 and 1926. Within this formalism, the central pivots of the theory are the physical observables (e.g., spin, electric charge, magnetic flux, position, momentum, etc.) represented by Hermitian operators (self-adjoint matrices) where the corresponding eigenvectors form a set of orthonormal basis for the Hilbert space.

---

# &nbsp; <a href="https://creativecommons.org/licenses/by/4.0/"><img valign="middle" src="https://img.icons8.com/copyright" width="50"></a> License

In [2]:
#@title Copyright 2022.
# This code is part of heisenberg-model.
#
# (C) Copyright NTNU QuCAI-Lab, 2022.
#
# This code is licensed under the Creative Commons Zero v1.0 Universal License. 
# You may obtain a copy of the License at https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/LICENSE.md.

# &nbsp; <a href="https://colab.research.google.com/"><img valign="middle" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="50"></a> Pip Install

* **Run the following cells only if you are running this Jupyter notebook outside the [heisenberg_model environment](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/environment.yml).**

Installing `Qiskit`, `NumPy`, and `pylatexenc`.

In [3]:
'''
# Installing a non-default library

!python3 -m pip install <library_name>

# Alternative 

!apt-get -qq install -y <library_name> && python3 -m pip install -U <library_name> 

# Upgrading an installed library

!python3 -m pip install -U --upgrade <library_name> 
'''

'\n# Installing a non-default library\n\n!python3 -m pip install <library_name>\n\n# Alternative \n\n!apt-get -qq install -y <library_name> && python3 -m pip install -U <library_name> \n\n# Upgrading an installed library\n\n!python3 -m pip install -U --upgrade <library_name> \n'

In [4]:
try:
  import pip, pkg_resources
  pkg_resources.require("pip>=21.1.3") # Latest version: 22.0.4
  print(pip.__version__)
except:
  import subprocess, sys
  cmd = "python3 -m pip install --upgrade pip"
  process = subprocess.Popen(cmd,shell=True,bufsize=1,stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,encoding='utf-8',errors='replace') 
  while True:
    out = process.stdout.readline()
    if out == '' and process.poll() is not None:
      break
    if out:
      print(out.strip(), flush=False)
      sys.stdout.flush()
  #raise

22.0.4


In [5]:
#!python3 -m pip install pip==version_number # To downgrade pip.
!pip --version

pip 22.0.4 from C:\Users\Qulab\.conda\envs\heisenberg-model\lib\site-packages\pip (python 3.7)



**NumPy:**

In [ ]:
!python3 -m pip install numpy==1.20.1

**Qiskit:**

In [ ]:
!python3 -m pip install qiskit==0.35.0

**pylatexenc**:

In [ ]:
# The 'pylatexenc' library is required to use 'MatplotlibDrawer'.

!python -m pip install pylatexenc==2.10

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/python" width="45" hspace="0px" vspace="0px"></a> Dependencies

- Importing modules.

In [6]:
import IPython
import qiskit
import numpy as np 
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library.standard_gates import U3Gate, RYGate
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi
from qiskit import BasicAer

In [7]:
### If using Google Colab, one needs to restart Colab Runtime after pip install. ###

try:
  import pkg_resources, watermark
  pkg_resources.require("watermark>=2.3.0") # Force watermark version.
  print(watermark.__version__)
except:
  import subprocess, sys
  if "google.colab" in sys.modules:
    cmd = "python3 -m pip install --upgrade watermark" # Upgrade watermak.

    process = subprocess.Popen(cmd,shell=True,bufsize=1,stdout=subprocess.PIPE, \
                              stderr=subprocess.STDOUT,encoding='utf-8',errors='replace') 
    while True: 
      out = process.stdout.readline() # The first line of the file.
      if out == '' and process.poll() is not None: # Run the loop until condition is True.
        break 
      if out:
        print(out.strip(), flush=False) # Removes leading and trailing empty spaces. 
        sys.stdout.flush()
    #raise # To raise the import error. Upgrade will be successful regardless.

2.3.0


In [8]:
# If you get hit by the error "No module named watermark", run this cell twice!

#%load_ext watermark
%reload_ext watermark
%watermark -a 'LucasCamponogaraViera' -gu 'QuCAI-Lab' -ws 'https://github.com/QuCAI-Lab/ibm2021-open-science-prize' -w -u -d -v -m -iv

Author: LucasCamponogaraViera

Github username: QuCAI-Lab

Website: https://github.com/QuCAI-Lab/ibm2021-open-science-prize

Last updated: 2022-04-27

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.32.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

numpy    : 1.20.1
qiskit   : 0.35.0
pip      : 22.0.4
watermark: 2.3.0
IPython  : 7.32.0

Watermark: 2.3.0



In [9]:
print(IPython.sys_info())

{'commit_hash': 'e84cee846',
 'commit_source': 'installation',
 'default_encoding': 'cp950',
 'ipython_path': 'C:\\Users\\Qulab\\.conda\\envs\\heisenberg-model\\lib\\site-packages\\IPython',
 'ipython_version': '7.32.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.19041-SP0',
 'sys_executable': 'C:\\Users\\Qulab\\.conda\\envs\\heisenberg-model\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit '
                '(AMD64)]'}


# Gate Algebra and Circuit Implementation

The bare minimum: examples with NumPY & Qiskit.

- Resource: see [Barenco et al.](https://arxiv.org/pdf/quant-ph/9503016.pdf) for a review on elementary gates and gate decomposition.

Quantum gates are all represented by Unitary matrices, but they might not be Hermitian. Most frequently used quantum gates are Hermitian:

- [The Hadamard gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html) (a.k.a H or superposition gate), induces a $\pi$ rotation about the $X+Z$ axis;
- [The CNOT gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html);
- [The SWAP gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.SwapGate.html);
- [The Toffoli gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CCXGate.html) (a.k.a CCNOT or deutsch);
- Fredkin (CSWAP) gate;
- All Pauli gates: [X](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#X:-bit-flip-gate), [Y](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#Y:-bit--and-phase-flip-gate), [Z](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#Z:-phase-flip-gate). 

However, there are non-hermitian gates such as:

- The phase gate [T](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TGate.html) (fourth-root of Pauli-$Z$);
- The [phase gate S](https://qiskit.org/documentation/stubs/qiskit.circuit.library.SGate.html) a.k.a as P gate (square-root of Pauli-$Z$).




A universal gate set is such to which any possible gate operation can be reduced to a finite sequence of gates from the set.

- The gate set containing the rotation operators $Rx(\theta)$, $Ry(\theta)$, $Rz(\theta)$, the phase shift gate $P(φ)$ and the CNOT gate is a common universal set of quantum gates.

- The Clifford + T gate set denoted {CNOT, H, S, T} is another widely used universal gate set.

Due to Gottesman–Knill theorem, the Clifford set can be efficiently simulated in a classical computer and, therefore, is the main gate set used to approximate a target quantum circuit in the [CDR error mitigation technique](https://mitiq.readthedocs.io/en/latest/guide/cdr-4-low-level.html). 

## Gate Operations

States $|0\rangle$ and $|1\rangle$ are in the $Z$-basis (a.k.a computational basis):

$$ Z |0\rangle =  |0\rangle.$$
$$ Z |1\rangle =  -|1\rangle.$$

This means that $|0\rangle$ and $|1\rangle$ are two eigenstates (orthonormal vectors) of the $Z$-gate, i.e, they form a basis set. Each one of these column vectors (kets) represent a classical bit. One can observe that the operation of the $Z$ gate on state $|1\rangle$ induces a phase represented by the minus sign.

Another basis other then the computational basis is the $X$-basis with eigenvectors $|+\rangle$ and $|-\rangle$ given by:

$$|\pm\rangle = (|0\rangle\pm|1\rangle)/\sqrt{2}.$$

One then has:

$$ X |0\rangle = |1\rangle.$$ 
$$X |1\rangle = |0\rangle.$$  

$$X |+\rangle = |+\rangle.$$
$$X |-\rangle = |-\rangle.$$

This means that $|+\rangle$ and $|-\rangle$ are two eigenstates of the $X$-gate, i.e, they form a basis set as already mentioned. Each one of these column vectors represent a linear combination (superposition) of two classical bits and, therefore, are named qubits. 

One can use the $H$ (Hadamard) gate to transform the state of the qubit between the $X$ and $Z$ bases:

$$H |0\rangle = |+\rangle.$$
$$H |1\rangle = |-\rangle.$$

## General $U3(\theta, \phi, \lambda)$ gate



Any arbitrary single qubit unitary U gate can be written in the form:

\begin{align}
    U &= e^{i\alpha}R_{\hat{n}}(\theta)=e^{i\alpha} e^{-i\theta \hat{n} \cdot \vec{\sigma}/2}\\
    &=e^{i\alpha} \left(cos\left(\frac{\theta}{2}\right)I_2-sin\left(\frac{\theta}{2}\right)(\hat{n}\cdot \vec{\sigma})\right),
\end{align}

where:
- $R_{\hat{n}}$ is the standard rotation gate;
-  $\hat{n}=(n_x, n_y, n_z)$ is a 3-dimensional unit row vector;
- $\vec{\sigma}=X \cdot n_x + Y \cdot n_y + Z \cdot n_z$ is a three component vector;
- X, Y, and Z are the 2x2 Pauli matrices;
- $a$ and $\theta \in \mathbb{R}$ are real-valued constants; 
- $i^2=-1$ is the imaginary unit.

Most used standard rotation gates:

\begin{equation}
R_x(\theta)= \exp(-iX\theta/2) = \begin{bmatrix} \cos(\theta/2) & -i\sin(\theta/2) \\ -i\sin(\theta/2) & \cos(\theta/2) \end{bmatrix},\\
R_y(\theta) =  \exp(-iY\theta/2)= \begin{bmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{bmatrix},\\
R_z(\theta) = \exp(-iZ\theta/2) = \begin{bmatrix} e^{-i\theta/2} & 0 \\0 & e^{i\theta/2}
\end{bmatrix}.
\end{equation}

The three parameter single qubit $U3$ gate reads:

\begin{equation}
U3(\theta, \phi, \lambda) = \begin{bmatrix} \cos(\theta/2) & -e^{i\lambda}\sin(\theta/2) \\ e^{i\phi}\sin(\theta/2) & -e^{i\lambda+i\phi}\cos(\theta/2) \end{bmatrix},
\end{equation}

where:

  - The angle $\lambda$ represents a phase.
  - The angle $\phi$ changes the coherence.
  - The angle $\theta$ performs the rotation around a given axis.

Particular gates:
  - $X = U3(\pi, 0, \pi)$.
  - $Y = U3(\pi, \pi/2, \pi/2)$.
  - $Z = U3(\pi,0,0)$.
  - $RX(\theta) = U3(\theta, -\pi/2,\pi/2)$.
  - $RY(\theta) = U3(\theta, 0,0)$.
  - $RZ(\phi) = U3(0, \phi,0)$.

## General $C_n^{j}(U_{2^m})$ gate with $n$ control qubits and $m$ target qubits



A generic single qubit rotation [$CU(\theta, \phi, \lambda, \gamma)$](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CUGate.html) gate can be written as:

\begin{equation}
I\otimes |0\rangle\langle 0\rangle + e^{i \gamma } U(\theta, \phi, \lambda, \gamma) \otimes |1 \rangle \langle 1|,
\end{equation}

where $e^{i \gamma }$ is a global phase.

Let $U_{2^m}$ be a $2^m$ x $2^m$ unitary matrix and $I_{2^m}$ be a $2^m$ x $2^m$ identity matrix, then a generic multicontrolled gate with $n$ control qubits and $m$ target qubits is given by:

\begin{equation}
C_n^{j}(U_{2^m}) = \sum_{i=0, i\neq j}^{2^n-1}I_{2^m} \otimes |i\rangle\langle i|+ U_{2^m} \otimes |j\rangle\langle j|.
\end{equation}

Particular cases (textbook convention for notations):

- $CX = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$.

- $CCX = I\otimes I\otimes |0\rangle\langle 0\rangle + CX \otimes |1 \rangle \langle 1|$.

- $CCCX = I\otimes I\otimes I\otimes |0\rangle\langle 0\rangle + CCX \otimes |1 \rangle \langle 1|$.

- $CU3(\theta, \phi, \lambda)$ where $CRY = CU3(\theta, 0,0)$.

In Qiskit's convention, for example, the CX gate writes: $|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X$.

## Gate Decomposition

### Single qubit gate decomposition

- A single qubit gate U can always be decomposed into:

\begin{equation}
U = e^{i\theta}AXBXC.
\end{equation}


### Two-qubit gate decomposition

A Control-U gate writes:

\begin{equation}
C(U) = \Phi_1(\theta)A_2C(X)B_2C(X)C_2,
\end{equation}
where
\begin{equation}
\Phi_1 = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{pmatrix} \otimes I_{2x2},
\end{equation}

is a phase gate applied to the first qubit and $X_2$, $B_2$ and $C_2$ are gates applied to the second qubit.

### $n$-qubit gate decomposition

- Lema for a general $n$-qubit unitary gate $U$ (a gate acting on a wire/qubit from a system of $n$ qubits):

If 
\begin{equation}
U = A_1 \cdot A_2 \cdot \cdots A_n.
\end{equation}

then
\begin{equation}
C(U) = C(A_1) \cdot C(A_2) \cdot \cdots C(A_n).
\end{equation}

## Implementations with NumPy and Qiskit  

- $U3(\theta, \phi, \lambda)$

In [7]:
'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import U3Gate
from qiskit.circuit import Parameter
'''

theta=Parameter('theta') 
phi=Parameter('phi') 
lam=Parameter('lam') 

qr=QuantumRegister(1)
qc = QuantumCircuit(qr)

gate=U3Gate(theta, phi, lam, label=None) # Uncomment this line if you need variables as placeholders.
#gate=U3Gate(np.pi, 0, 0, label=None) # Comment this line if you need variables as placeholders.
qc.append(gate,qr)
qc.draw()

┌───────────────────┐
q0: ┤ U3(theta,phi,lam) ├
    └───────────────────┘

- $CX = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$

In [8]:
# Qiskit Implementation of CX gate.

'''
import qiskit.quantum_info as qi
'''

qr=QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.cnot(0,1)

print(qc.draw())
qi.Operator(qc)

           
q1_0: ──■──
      ┌─┴─┐
q1_1: ┤ X ├
      └───┘


Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

In [9]:
# NumPy Implementation of CX gate.

'''
import numpy as np
'''

psi_0 = np.array([1, 0])
psi_1 = np.array([0, 1])

outer0 = np.outer(psi_0, psi_0)
outer1 = np.outer(psi_1, psi_1)

id=np.identity(2)

X=np.array([[0, 1], [1, 0]]) 

cx = np.kron(id, outer0) + np.kron(X, outer1)
cx

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

- $CCX = I\otimes I\otimes |0\rangle\langle 0\rangle + CX \otimes |1 \rangle \langle 1|$

In [10]:
# Qiskit Implementation of CCX gate.

'''
from qiskit import QuantumCircuit, QuantumRegister,transpile
from qiskit import BasicAer
'''

backend = BasicAer.get_backend('unitary_simulator')
q = QuantumRegister(3)
qc = QuantumCircuit(q)
qc.ccx(q[0], q[1], q[2])

print(qc.draw())
job = backend.run(transpile(qc, backend))
job.result().get_unitary(qc, decimals=3)

           
q2_0: ──■──
        │  
q2_1: ──■──
      ┌─┴─┐
q2_2: ┤ X ├
      └───┘


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [11]:
# NumPy Implementation of CCX gate.

id4 = np.kron(id, id)
ccx = np.kron(id4, outer0) + np.kron(cx, outer1)
ccx

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]])

- $CCCX = I\otimes I\otimes I\otimes |0\rangle\langle 0\rangle + CCX \otimes |1 \rangle \langle 1|$

In [12]:
# NumPy Implementation of CCCX gate.

cccx = np.kron(np.identity(8), outer0) + np.kron(ccx, outer1)
cccx

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

- $CCRY$

In [13]:
# Qiskit Implementation of CCRY gate.

'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
import qiskit.quantum_info as qi
'''

theta=np.pi
qr=QuantumRegister(3)
qc = QuantumCircuit(qr)
control=RYGate(theta).control(2)
qc.append(control,qr)

print(qc.draw())
qi.Operator(qc)

                
q12_0: ────■────
           │    
q12_1: ────■────
       ┌───┴───┐
q12_2: ┤ Ry(π) ├
       └───────┘


Operator([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
           -1.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,
            0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j]],
         input_dims=(2, 2, 2), output_dims=(2, 2, 2))

- $CCCRY$ (16x16 matrix)

In [14]:
# Qiskit Implementation of CCCRY gate.

'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
import qiskit.quantum_info as qi
'''

theta=np.pi
qr=QuantumRegister(4)
qc = QuantumCircuit(qr)
control=RYGate(theta).control(3)
qc.append(control,qr)

print(qc.draw())
qi.Operator(qc)

                
q13_0: ────■────
           │    
q13_1: ────■────
           │    
q13_2: ────■────
       ┌───┴───┐
q13_3: ┤ Ry(π) ├
       └───────┘


Operator([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  1.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+0

# The Heisenberg XXX Spin-1/2 Lattice Model for $N=3$ Three Particles


The [Quantum Heisenberg XXX spin-1/2 model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model#XXX_model) is a particular case of the general [XYZ Quantum Heisenberg Model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model) when the coupling coefficients are $J_x = J_y = J_z \doteq J$. The particle interaction of the aforementioned spin model for a system of $N$ quantum spin-1/2 particles arranged in a 1D (one-dimensional) Markov chain, where each particle is represented by a qubit, is given by the following `Hamiltonian operator`:

\begin{eqnarray}
h_{\text{heis}} &=& \sum_{\langle jk \rangle =0}^{N-1} J \left(\sigma_x^{(j)}\sigma_x^{(k)} + \sigma_y^{(j)}\sigma_y^{(k)} + \sigma_z^{(j)}\sigma_z^{(k)}\right).
\end{eqnarray}
Where: 
- $J$ is a real-valued [coupling](https://en.m.wikipedia.org/wiki/Coupling_constant) constant that determines the interaction strength.
- $\langle jk \rangle$ specifies interaction only between nearest neighbor qubits $j$ and $k$ ($j,k \in \{0,1,2\}$).
- $\otimes$ denotes the [tensor product](https://en.wikipedia.org/wiki/Tensor_product#Tensor_product_of_linear_maps) symbol.

<br></br>
In classical mechanics, the Hamiltonian $H$ corresponds to the sum of kinetic and potential energy of a **time-independent holonomic and monogenic system** \[[Goldstein, H.](#Ref)]). In quantum mechanics, the Hamiltonian is promoted to a so-called Hermitian operator $\hat{H}$ represented by a self-adjoint complex matrix within the matrix mechanics formalism.

The Pauli operators in this 1D spin lattice satisfy the following Lie algebra for the (anti-)commutation relations:


\begin{eqnarray}
\text{commutator: } \{\sigma_a^{j}, \sigma_b^{k} \} = 2\delta_{jk}\delta_{ab}\sigma_0^{j},\\
\end{eqnarray}
\begin{eqnarray}
\text{anti-commutator: }
[\sigma_a^{j}, \sigma_b^{k}] = 2i\delta_{jk}\epsilon_{abc}\sigma_c^{j},
\end{eqnarray}

Where:

- $[\hat{A},\hat{B}]$ denotes the commutator between operators $\hat{A}$ and $\hat{B}$:
\begin{align}
[\hat{A},\hat{B}]\doteq\hat{A}\hat{B} - \hat{B}\hat{A}.
\end{align}
- $i=\sqrt{-1}$ denotes the imaginary unit.
- $\epsilon_{abc}$ denotes the normalized Levi-Civita symbol ($\epsilon_{xyz}=1$) with $a, b, c \in \{x,y,z\}$:

\begin{align*}
\varepsilon_{abc} \doteq
\begin{cases}
+1 & \text{for even permutation of }(a,b,c). \\
-1 & \text{for odd permutation of } (a,b,c).  \\
\;\;\,0 & \text{for any repeated index}.
\end{cases}
\end{align*}

- $\delta_{jk}$ denotes the Kronecker delta:

\begin{eqnarray}
\delta_{jk} \doteq \begin{cases}
0, & \mbox{if } j \ne k, \\
1, & \mbox{if } j=k. \end{cases}
\end{eqnarray}
- $\{\sigma_x,\sigma_y,\sigma_z\}$ denotes the $SU(2)$ set of [Pauli operators](https://en.wikipedia.org/wiki/Pauli_matrices) (2x2 complex Hermitian and Unitary matrices sharing equal eigenvalues of $\pm 1$).

For the particular case of a system of $N=3$ spin-1/2 particles arranged in a line, and setting $J=1$, one then has:

\begin{eqnarray}
h_{\text{heis}} &=& \sigma_x^{(0)}\otimes\sigma_x^{(1)}\otimes I^{(2)} + I^{(0)} \otimes\sigma_x^{(1)}\otimes\sigma_x^{(2)} + \sigma_y^{(0)}\otimes\sigma_y^{(1)}\otimes I^{(2)} + I^{(0)} \otimes \sigma_y^{(1)}\otimes\sigma_y^{(2)} + I^{(0)} \otimes\sigma_z^{(0)}\otimes\sigma_z^{(1)} + I^{(0)}\otimes\sigma_z^{(1)}\otimes\sigma_z^{(2)}.
\end{eqnarray}

However, the convention adopted by the quantum community is the following shorthand notation (omitting the tensor product ($\otimes$) and the 2x2 identity matrix $I$):

\begin{eqnarray}
h_{\text{heis3}} = \sigma_x^{(0)}\sigma_x^{(1)} + \sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(0)}\sigma_y^{(1)} + \sigma_y^{(1)}\sigma_y^{(2)} + \sigma_z^{(0)}\sigma_z^{(1)} + \sigma_z^{(1)}\sigma_z^{(2)}.
\end{eqnarray}

# Unitary Evolution Operator

## **System's Dynamics**

In quantum mechanics, the dynamics of a system of particles in the position space is determined by the linear partial differential Schrödinger equation:

\begin{eqnarray}
i \hbar \frac{\partial \psi(\vec{r}, t)}{\partial t} &= \hat{H}(t)\psi(\vec{r}, t),
\end{eqnarray}

where $\psi(\vec{r}, t)$ is the wavefunction containing the information about the particle's dynamics in the *position space*. In the Hilbert state (vector) space, solution for $\hat{U}(t)$ comes from the time-dependent Schrödinger equation:

\begin{eqnarray}
i \hbar\frac{d}{dt}|\psi(t)\rangle= \hat{H} |\psi(t)\rangle.
\end{eqnarray}

Solutions for the above equations depends mostly on the characteristic of the Hamiltonian. In the particular case of a **time-independent Hamiltonian** $\hat{H}(t)\doteq \hat{H}$, one would adopt the following convenient separation of variables: $\psi(\vec{r},t)=\phi(\vec{r})\chi(t)$. In the state space, however, using Dirac's bra-ket notation, one has: 

\begin{equation} 
i\hbar \frac{d}{dt}(\hat{U}(t) |\psi(0)\rangle)=\hat{H}(t) (\hat{U}(t) |\psi(0)\rangle).
\end{equation}  

which is the time-dependent Schrödinger equation of the *time evolution operator*, with initial condition $U(t=0)=I$. One then has:

\begin{equation} 
 \frac{d}{dt} \hat{U}(t) = \frac{-i}{\hbar}\hat{H}\hat{U}(t).
\end{equation}  

Using the fact that any Unitary operator can be constructed by means of some Hermitian operator ($\hat{H}=\hat{H}^{\dagger}$), one gets:

\begin{eqnarray}
\hat{U}=e^{-i\gamma\hat{H}}.
\end{eqnarray}

To show the above is true we prove its unitarity:

\begin{eqnarray}
\hat{U}^{\dagger}&=&(e^{-i\gamma\hat{H}})^{\dagger}=e^{i\gamma\hat{H}^{\dagger}} = e^{i\gamma\hat{H}}, \\
&\implies&
\hat{U}\hat{U}^{\dagger}=e^{-i\gamma\hat{H}}e^{i\gamma\hat{H}}=I.
\end{eqnarray}

And the closed-form solution thus becomes:

\begin{eqnarray}
\hat{U}(t) = e^{-i\hat{H}t / \hbar}.
\end{eqnarray}

One can check this result by expanding the exponential function in Taylor series around the point $x_0=0$ (Maclaurin series) and differentiating term by term. The Taylor series is defined as:

\begin{eqnarray}
g(x)= \sum_{n=0}^{\infty} f^{n}(x_0)\frac{(x-x_0)^n}{n!},
\end{eqnarray}

where $f^{n}$ denotes the n-th derivative of $f$. For $f(x)=f^{n}(x)=e^{x}$ and $e^{x_0=0}=1$, one get:

\begin{eqnarray}
e^{i\gamma\hat{H}} = \sum_{n=0}^{\infty} \frac{({i\gamma\hat{H}})^n}{n!}.
\end{eqnarray}

So that:

\begin{eqnarray}
i\hbar \frac{d}{dt} (e^{-i\hat{H}t / \hbar})= i\hbar\frac{d}{dt} \left( 1 - i\hat{H}t/\hbar - \frac{\hat{H}^2 t^2}{2!\hbar^2}  + \cdots + \frac{(i\hat{H}t)^n}{n!\hbar^n} \approx \prod_j e^{-i\hat{H}_jt / \hbar}\right) = \hat{H}.
\end{eqnarray}

# Time Evolution of the Initial State $|110\rangle$ under $h_{heis3}$ According to $U_{\text{heis3}}(\pi)$


**To see how the state of the quantum system envolves from the initial state $|110\rangle$ to the final state $U_{\text{Heis3}}(t=\pi)|110\rangle = |\psi(\pi)\rangle$ with probability $|\langle 110 | U_{\text{Heis3}}(\pi) |110\rangle|^2$, we take a quick step back to review the first and second postulates of quantum mechanics in the ubiquitous Dirac bra-ket notation.**

## **Postulate 1 (State Space)**

Any isolated (close) physical system is embedded in a complex $d$-dimensional Hilbert space $\mathcal{H}^d$ known as the *state space* of the system endowed with an inner product $(\cdot,\cdot) \doteq \langle \cdot | \cdot \rangle$. The system is completely described by a normalized *state vector* $|\psi\rangle \in \mathcal{H}^d$ which is a unit vector in the system's state space. In the orthonormal basis set $\{|o_j\rangle\}_{j=1}^d$, the state vector reads
\begin{eqnarray}
|\psi\rangle \doteq \sum_{j=1}^d c_j|o_j\rangle,
\end{eqnarray}
where $c_j=\langle o_j|\psi\rangle$ denotes the probability amplitude associated with the preparation of an eigenstate $|o_j\rangle$.



## **Postulate 2 (Evolution)**

A **closed quantum system** evolves over time under a *unitary transformation* such that, in the *Schrödinger picture formalism*, the initial state $|\psi(0)\rangle$ of the system evolves after a time $t$ to:

\begin{equation} 
|\psi(t)\rangle=\hat{U}(t)|\psi (0)\rangle,
\end{equation}  
according to the action of the propagator $\hat{U}(t)$ (a Unitary operator). For the particular case of closed systems, the evolution operator $\hat{U}_t$ is Unitary, however, open systems (systems interacting with its surroundings), in general, have non-Unitary operators.







## **Probability Over Time $t$**

Consider a physical observable represented by a Hermitian ($\hat{\mathcal{O}}=\hat{\mathcal{O}}^{\dagger}$) and, therefore, Normal ($ \hat{\mathcal{O}}\hat{\mathcal{O}}^{\dagger}=\hat{\mathcal{O}}^{\dagger}\hat{\mathcal{O}}$) matrix with real eigenvalues and diagonal representation (spectral decomposition):
\begin{eqnarray}
\hat{\mathcal{O}} = \sum_{j=1}^d o_j P_{o_j}= \sum_{j=1}^d o_j |o_j\rangle \langle o_j|,
\end{eqnarray}
where $P_{o_j}=|o_j\rangle \langle o_j|$ is the projector onto the eigenspace of the observable $\hat{\mathcal{O}}$ in some $d$-dimensional orthonormal basis set $\{|o_j \rangle\}_{j=1}^d$ ($\langle o_j|o_k\rangle=\delta_{jk}$) of eigenvectors $|o_j \rangle$ with eigenvalues $o_j$.

For a given quantum system prepared in the state $|\psi\rangle \doteq \sum_{j=1}^d c_j|o_j\rangle$, a projective measurement (a.k.a von Neumann measurement) entails the following conditional probability for finding the system in the eigenstate $|o_j\rangle$ with eigenvalue $o_j$ upon measurement (wave function collapse):

\begin{eqnarray} 
Pr(o_j|\psi\rangle)&=&\langle \psi | P^{\dagger}_{o_j} P_{o_j} |\psi \rangle \\ 
&=& \langle \psi | P_{o_j}^2 |\psi \rangle\\
&=& \langle \psi| (|o_j\rangle \langle o_j|o_j\rangle \langle o_j|)| \psi \rangle\\
&=&\delta_{jj}\langle\psi|o_j\rangle \langle o_j |\psi\rangle \\ 
&=& |\langle \psi|o_j \rangle|^2=|\langle o_j|\psi \rangle|^2\\
&=& \left|\langle o_j|\sum_{k=1}^d c_k |o_k\rangle\right|^2\\
&=&  \left|\sum_{k=1}^d c_k \langle o_j|o_k\rangle\right|^2\\
&=&\left|\sum_{k=1}^d c_k \delta_{jk} \right|^2 = |c_j|^2.
\end{eqnarray} 

This result is also known as Born's rule:

\begin{eqnarray}
P(|o_j\rangle) = |\langle o_j |\psi \rangle|^2.
\end{eqnarray}

<br></br>
If the system has evolved from the initial state $|\psi(0)\rangle$ to the final state $|\psi(t)\rangle$, then:

\begin{eqnarray}
P_t(|o_j\rangle) &=& |\langle o_j | \hat{U}(t) |\psi(0)\rangle|^2 \\
&=& |\langle o_j |\psi(t)\rangle|^2.
\end{eqnarray}

<br></br>
Therefore, the probability of measuring the initial state $|\psi(0)\rangle=|110\rangle$ over a time window $t=\pi$ under the **time-independent Hamiltonian** XXX Heisemberg Hamiltonian is:

\begin{eqnarray}
P_{t=\pi}(|110\rangle) &=& |\langle 110 | u_{heis3}(t=\pi) |110\rangle|^2 \\
&=& |\langle 110 |e^{-ih_{heis3}\pi / \hbar} |110\rangle|^2 \\
&=& |\langle u_{heis3} \rangle|^2.
\end{eqnarray}

Q.E.D.

The last equation is also known to be the modulus squared of the expectation value $\langle \hat{U}(t) \rangle$ of the unitary evolution operator. To see this, we consider the expectation value of a general observable $\hat{\mathcal{O}}$:
\begin{align}
\langle \hat{\mathcal{O}}\rangle &=\sum_{j=1}^d o_j  \langle \psi |o_j \rangle \langle o_j|\psi \rangle \\
&= \langle \psi | \left(\sum_{j=1}^d o_j |o_j \rangle \langle o_j| \right) |\psi \rangle\\
&= \langle \psi | \left(\sum_{j=1}^d o_j P_{o_j} \right) |\psi \rangle\\
&=\langle \psi |\hat{\mathcal{O}}|\psi\rangle.
\end{align}

Upon evolution of the system's state vector ($\hat{U}(t)|\psi\rangle$), the expected value becomes:

\begin{align}
\langle \psi |U^{\dagger}\hat{\mathcal{O}}\hat{U}|\psi\rangle.
\end{align}

Thus, when $\hat{\mathcal{O}} = \hat{U}(t)$ one has:
\begin{align}
\langle \psi |U^{\dagger}(t)\hat{U}(t)\hat{U}(t)|\psi\rangle = \langle \psi |\hat{U}(t)|\psi\rangle = \langle \hat{U}(t) \rangle.
\end{align}

Q.E.D.

# The Trotter-Susuki formula

Consider the following decomposition for a general Hamiltonian $H$:

\begin{align}
H = H_a + H_b.
\end{align}

If the reduced Hamiltonians commute, i.e, $[H_{a},H_{b}]=\mathbb{O}$ then the evolution operator is product of the reduced operators:

\begin{align}
U = e^{-iHt/ħ} = e^{-iH_{a}t/\hbar-iH_{b}t/\hbar} = e^{-iH_{a}t/\hbar}e^{-iH_{b}t/\hbar} = U_{a}U_{b} = U_{b}U_{a}.
\end{align}

**Theorem (simultaneous diagonalization):**

*Suppose $A$ and $B$ are two Hermitian operators. Then $[A, B] = AB-BA$ is equal to the zero matrix (they commute) if and only if there exists an orthonormal basis such that both $A$ and $B$ are diagonal with respect to that basis, i.e, they share the same basis. Therefore, $A$ and $B$ are simultaneously diagonalizable with different eigenvalues.*





In the case where $[H_a^{(0,1)},H_b^{(1,2)}]\ne\mathbb{O}$, one cannot simply write the evolution operator $U$ as a product of evolution operators, therefore, another method must be adopted. There are several techniques to approximate a time Unitary evolution $\hat{U}=e^{-i\hat{H}t/\hbar}$ into gate operations for quantum simulation, to name a few:

1. Trotter-Susuki formula.
2. Randomized evolution (Qdrift, density matrix exponentiation).
3. Linear combination of unitaries.
4. Quantum Walks (Qubitization).



The Trotter-Susuki formula is defined as (setting $\hbar\equiv1$):

\begin{align}
U_{Trotter}(t) = e^{-it\Big(H_a + H_b\Big)} \equiv \lim_{N\rightarrow\infty}\Big(e^{-it H_a/N}e^{-it H_b /N}\Big)^{N}.
\end{align}

Advantages of using Trotterization:

- The algorithm is ancilla-free.

- It leverages the commutative property of the component Hamiltonians to achieve efficient simulation.

- It preserves the locality property of some unidimensional systems with nearest neighbor interactions enabling quantum simulation speed up.







# Decomposition of $U_{\text{Heis3}}(t)$ using Trotterization

Consider the following decomposition for the Hamiltonian $H_{heis3}$:

\begin{align}
H_{heis3} = H_a^{(0,1)} + H_b^{(1,2)},
\end{align}
where
\begin{align}
H_a^{(0,1)}  =\big(X^{(0)} \otimes X^{(1)} + Y^{(0)}\otimes Y^{(1)} + Z^{(0)}\otimes Z^{(1)}\big)\otimes I^{(2)}, \\
H_b^{(1,2)} =I^{(0)}\otimes \big(X^{(1)}\otimes X^{(2)} +  Y^{(1)}\otimes Y^{(2)} +  Z^{(1)}\otimes Z^{(2)}\big),
\end{align}

We note that the pair of operators in the exponential of $U_{Trotter}(t)$ commute, i.e,

$$[X\otimes X,Y\otimes Y] = [X\otimes X,Z\otimes Z] = [Y\otimes Y,Z\otimes Z] =\mathbb{O}.$$

One can verify this is true, as follows ($j\ne k$):

\begin{eqnarray}
[\sigma_{j}\otimes\sigma_{j},\sigma_{k}\otimes\sigma_{k}] & = (\sigma_{j}\otimes\sigma_{j}) \cdot (\sigma_{k}\otimes\sigma_{k}) - (\sigma_{k}\otimes\sigma_{k}) \cdot (\sigma_{j}\otimes\sigma_{j}),
\end{eqnarray}

using $(A\otimes B)\cdot(C\otimes D)=AC\otimes BD$, one then has

\begin{eqnarray}
=\sigma_{j}\sigma_{k}\otimes\sigma_{j}\sigma_{k} - \sigma_{k}\sigma_{j}\otimes\sigma_{k}\sigma_{j} 
\end{eqnarray}

and given the $\mathfrak{su}(2)$ Lie algebra $\sigma_{j}\sigma_{k} = \sigma_{0}\delta_{jk}+i\epsilon_{jkl}\sigma_{l}$, it becomes

\begin{eqnarray}
&=& i\epsilon_{jkl}\sigma_{l}\otimes i\epsilon_{jkl}\sigma_{l} -i\epsilon_{kjl}\sigma_{l}\otimes i\epsilon_{kjl}\sigma_{l} \\
&=& i^{2}\epsilon_{jkl}^{2}\sigma_{l}\otimes\sigma_{l} -i^{2}\epsilon_{kjl}^{2}\sigma_{l}\otimes \sigma_{l} \\
&=& i^{2}\epsilon_{jkl}^{2}\sigma_{l}\otimes\sigma_{l} -i^{2}(-\epsilon_{jkl})^{2}\sigma_{l}\otimes \sigma_{l} \\
&=& \mathbb{O}.
\end{eqnarray}


With the above result in hands, the exponential $e^{-itH_a^{01}} $ can be written in the form:

\begin{eqnarray}
e^{-itH_a^{01}} &=& e^{-it(X\otimes X+ Y\otimes Y + Z\otimes Z)\otimes I} \\ 
&=& e^{-it(X\otimes X\otimes I + Y\otimes Y\otimes I + Z\otimes Z\otimes I)} \\
&=& e^{-itX\otimes X\otimes I}e^{-itY\otimes Y\otimes I}e^{-itZ\otimes Z\otimes I},
\end{eqnarray}

which after using $e^{A\otimes I} = e^{A}\otimes I$, one gets

\begin{eqnarray}
e^{-itH_a^{01}}&=& \Big(e^{-itX\otimes X}\otimes I\Big)\Big(e^{-itY\otimes Y}\otimes I\Big)\Big(e^{-itZ\otimes Z}\otimes I\Big), \\
&=& \Big(e^{-itX\otimes X}\Big)\Big(e^{-itY\otimes Y}\Big)\Big(e^{-itZ\otimes Z}\Big)\otimes I.
\end{eqnarray}

And equivalenty for $e^{-itH_b^{12}}$:

\begin{eqnarray}
e^{-itH_b^{12}} &= I\otimes\Big( e^{-itX\otimes X}\Big)\Big( e^{-itY\otimes Y}\Big)\Big( e^{-itZ\otimes Z}\Big).
\end{eqnarray}

The unitary evolution operator for the decomposition of $H_{heis3}$ according to the Trotter-Suzuki fomula thus becomes:

\begin{eqnarray}
U_{\text{Heis3}}(t) &=& e^{-itH_{heis3}} = e^{-it\Big(H_a^{(0,1)} + H_b^{(1,2)}\Big)} =  \lim_{N\rightarrow\infty}\Big(e^{-it H_a^{(0,1)}/N}e^{-it H_b^{(1,2)} /N}\Big)^{N}\\
&=& \lim_{N\rightarrow\infty}\left(\Big(e^{-it X\otimes X/N}e^{-it Y\otimes Y/N}e^{-it Z\otimes Z/N}\otimes I\Big)\Big(I\otimes e^{-it X\otimes X/N}e^{-it Y\otimes Y/N}e^{-it Z \otimes Z/N}\Big)\right)^{N}.
\end{eqnarray}

# Transpilation of $U_{\text{Heis3}}(t)$ Into Quantum Gates

The outline of the Trotterization algorithm a.k.a as product-formula method or splitting method is as follows:

**1.** Write the Hamiltonian operator $\hat{H}$ of the system of interest in the form:

\begin{eqnarray}
\hat{H}= \sum_{\gamma=1}^{\Gamma} \hat{H}_{\gamma},
\end{eqnarray}
where $\hat{H}_{\gamma}$ is itself a Hermitian operator.

**2.** Write the unitary evolution operator $U$ in terms of product of exponentials using the first-order Lie-Trotter formula defined as:

\begin{eqnarray}
\mathcal{L}_1(t) = e^{-itH_{1}} \cdots e^{-itH_{\Gamma}} = e^{-itH} + O(t^2),
\end{eqnarray}
where $O(t^2)$ is the trotter error.

**3.** Perform $e^{-it\hat{H}}$ up to some error $\epsilon$ according to the spectral norm:

\begin{eqnarray}
||U-e^{-it\hat{H}}|| \leq \epsilon.
\end{eqnarray}

**4.** Define the total cost of the simulation as:
\begin{eqnarray}
\text{total cost} = \text{no. of steps } \times \text{ cost}/\text{step}.
\end{eqnarray}

In step 3, each exponential in the evolution operator $U_{\text{Heis3}}(t)$ is transpiled into a quantum gate.

## The $e^{-itZZ}$ gate

IBM supplementary material provides the following quantum circuit for the $e^{-itZZ}$ gate:

In [15]:
# e^{(-itZZ)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
#qc.draw(output='mpl') # Requires 'pip install pylatexenc' library to use 'MatplotlibDrawer'.
qc.draw()

q_0: ──■───────────────■──
     ┌─┴─┐┌─────────┐┌─┴─┐
q_1: ┤ X ├┤ Rz(2*t) ├┤ X ├
     └───┘└─────────┘└───┘

**Verification:**

We first note that $RZ(2t) = e^{-itZ}$ (see chapter "Gate Algebra and Circuit Implementation"). One then has:

\begin{eqnarray}
CX^{01}\big(I\otimes RZ(2t)\big)CX^{01} =
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}\\ 
\end{eqnarray}

From the aforementioned chapter, using the equation for a generic CU gate one can write $CX^{01} = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$. However, in Qiskit’s convention this gate writes $CX^{01}=|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X$. One then has:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}=(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X)\big(I\otimes e^{-itZ}\big)\big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big).
\end{eqnarray}

From here, one should note that since each term between a tensor product is a matrix, the distributivity property for matrix product with respect to matrix addition yelds:

$$\big(I\otimes e^{-itZ}\big)\big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big) = \big(I\otimes e^{-itZ}\big) \big(|0\rangle\langle 0|\otimes I\big) + \big(I\otimes e^{-itZ}\big)\big(|1\rangle\langle 1|\otimes X \big),$$

Using $(A\otimes B)\cdot(C\otimes D)=A\cdot C\otimes B\cdot D$, the last term becomes:

$$\big(I \cdot |0\rangle\langle 0|\otimes e^{-itZ} + I \cdot |1\rangle\langle 1|\otimes e^{-itZ}X \big) = \big(|0\rangle\langle 0|\otimes e^{-itZ} + |1\rangle\langle 1|\otimes e^{-itZ}X \big) .$$ 

The final steps carry on as follows:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01} &=& \big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big)\big(|0\rangle\langle 0|\otimes e^{-itZ} +|1\rangle\langle 1|\otimes e^{-itZ}X \big) \\
&=& |0\rangle\langle 0|\otimes e^{-itZ} +|1\rangle\langle 1|\otimes Xe^{-itZ}X \\ 
&=& |0\rangle\langle 0|\otimes \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big) +|1\rangle\langle 1|\otimes X \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big)X,
\end{eqnarray}

and noting the operation $X|0\rangle = |1\rangle$, one finally gets:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}&=& |0\rangle\langle 0|\otimes \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big) +|1\rangle\langle 1|\otimes \big(e^{-it}|1\rangle\langle 1|+e^{it}|0\rangle\langle 0|\big) \\
&=& e^{-it}|00\rangle\langle 00| + e^{it}|01\rangle\langle 01| + e^{-it}|11\rangle\langle 11| + e^{it}|10\rangle\langle 10| \\
&=& e^{-itZZ}.
\end{eqnarray}


## The $e^{-itXX}$ gate

IBM supplementary material provides the following quantum circuit for $e^{-itXX}$:

In [16]:
# e^{(-itXX)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.ry(np.pi/2,[0,1])
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
qc.ry(-np.pi/2,[0,1])

qc.draw()

┌─────────┐                     ┌──────────┐
q_0: ┤ Ry(π/2) ├──■───────────────■──┤ Ry(-π/2) ├
     ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐├──────────┤
q_1: ┤ Ry(π/2) ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ Ry(-π/2) ├
     └─────────┘└───┘└─────────┘└───┘└──────────┘

From the fact that $H|0\rangle = R_y(\pi/2)|0\rangle = |+\rangle = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$, one equivalent quantum circuit is:

In [17]:
# Equivalent circuit:

qc = QuantumCircuit(2)
qc.h([0,1])
qc.cx(0, 1)
qc.rz(2 * t, 1)
qc.cx(0, 1)
qc.h([0,1])
qc.draw()

┌───┐                     ┌───┐
q_0: ┤ H ├──■───────────────■──┤ H ├
     ├───┤┌─┴─┐┌─────────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ H ├
     └───┘└───┘└─────────┘└───┘└───┘

**Verification:**


## The $e^{-itYY}$ gate

IBM supplementary material provides the following quantum circuit for $e^{-itYY}$:

In [18]:
# e^{(-itZZ)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.rx(np.pi/2,[0,1])
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
qc.rx(-np.pi/2,[0,1])

qc.draw()

┌─────────┐                     ┌──────────┐
q_0: ┤ Rx(π/2) ├──■───────────────■──┤ Rx(-π/2) ├
     ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐├──────────┤
q_1: ┤ Rx(π/2) ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ Rx(-π/2) ├
     └─────────┘└───┘└─────────┘└───┘└──────────┘

From the fact that $ SH|0\rangle = R_x(\pi/2)|0\rangle) = \frac{1}{\sqrt{2}}(|0\rangle+i|1\rangle) $, one equivalent quantum circuit is:

In [19]:
# Equivalent circuit:

qc = QuantumCircuit(2)
qc.sdg([0,1])
qc.h([0,1])
qc.cx(0, 1)
qc.rz(2 * t, 1)
qc.cx(0, 1)
qc.h([0,1])
qc.s([0,1])
qc.draw()

┌─────┐┌───┐                     ┌───┐┌───┐
q_0: ┤ Sdg ├┤ H ├──■───────────────■──┤ H ├┤ S ├
     ├─────┤├───┤┌─┴─┐┌─────────┐┌─┴─┐├───┤├───┤
q_1: ┤ Sdg ├┤ H ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ H ├┤ S ├
     └─────┘└───┘└───┘└─────────┘└───┘└───┘└───┘

**Verification:**


# State Fidelity

In noisy environments, the evolved quantum state must be represented by a mixed density operator, since noise creates mixed states. The fidelity between two mixed density operators (density matrices) $\rho$ and $\sigma$ is:

\begin{align}
F(\rho,\sigma) = \left(tr \sqrt{\rho^{1/2} \sigma \rho^{1/2}}\right)^2,
\end{align}

with $\rho= p_j\sum_j|\psi_{j}\rangle \langle \psi_{j}|$ and $\sigma=p_k\sum_k|\psi_{k}\rangle \langle \psi_{k}|$, where $p_j$ and $p_k$ are the corresponding (eigenvalue) probability distributions.

If one of the states is a pure state ($\rho=|\psi_{\rho}\rangle \langle \psi_{\rho}|$), the above result boils down to the fidelity between a pure and a mixed state:

\begin{align}
F(\sigma, |\psi_{\rho}\rangle) = \langle\psi_{\rho}|\sigma|\psi_{\rho}\rangle = \sum_{j,k=0}^{d-1}\psi_{j}^{*}\sigma_{j,k}\psi_{k}.
\end{align}

In a noise-free environment, both the evolved state and the target states are pure states, the state fidelity thus becomes:

\begin{align}
F(\rho, \sigma) = |\langle\psi_{\rho}|\psi_{\sigma}\rangle|^2.
\end{align}

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/book" width="45" hspace="0px" vspace="0px"></a> References<a name="Ref" />

\[1] Nielsen MA, Chuang IL. 2010. Quantum Computation and Quantum Information. New York: [Cambridge Univ. Press.](https://doi.org/10.1017/CBO9780511976667) 10th Anniv. Ed. Chapter 4.7.2: The quantum simulation algorithm, pg. 207, Theorem 4.3: (Trotter formula). Chapter 9.2.2: Fidelity, pg. 409.
  
\[2] Griffiths, David J., and Darrell F. Schroeter. Introduction to quantum mechanics. Cambridge University Press, 2018.

\[3] [ibmq-qsim-challenge.ipynb](https://github.com/qiskit-community/open-science-prize-2021/blob/main/ibmq-qsim-challenge.ipynb).

\[4] [ibmq-qsim-sup-mat.ipynb](https://github.com/qiskit-community/open-science-prize-2021/blob/main/ibmq-qsim-sup-mat.ipynb).